In [1]:
from Channel_Aligned_Robust_Dual_Transformer import CARD

In [2]:
model = CARD.Transformer(50, 3, 2, 3, 1, 1024, 8, 64, 0.9, 3)

In [9]:
CARD.model_builder(model, train[:1])

In [11]:
model.load_weights("C:/Users/Nguyen Phat/AI-Trading-Assistant/AI_Trading_Assistant_Model/model_weights/best_model_new_odd.h5")

In [12]:
model(train[:1])

<tf.Tensor: shape=(1, 2, 3), dtype=float32, numpy=
array([[[149.20158, 132.84094, 157.0693 ],
        [148.26234, 134.18405, 115.97041]]], dtype=float32)>

In [13]:
model.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dual_transformer_block (Du  multiple                  4135450   
 alTransformerBlock)                                             
                                                                 
 tensor2_seq (Tensor2Seq)    multiple                  9411      
                                                                 
Total params: 4144861 (15.81 MB)
Trainable params: 4144765 (15.81 MB)
Non-trainable params: 96 (384.00 Byte)
_________________________________________________________________


In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler


In [4]:
df = yf.download("Meta", start="2018-11-01", end="2023-10-1", interval="1d")
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-11-01,151.520004,152.750000,149.350006,151.750000,151.750000,25640800
2018-11-02,151.800003,154.130005,148.960007,150.350006,150.350006,24708700
2018-11-05,150.100006,150.190002,147.440002,148.679993,148.679993,15971200
2018-11-06,149.309998,150.970001,148.000000,149.940002,149.940002,16667100
2018-11-07,151.570007,153.009995,149.830002,151.529999,151.529999,21877400
...,...,...,...,...,...,...
2023-09-25,295.640015,300.950012,293.700012,300.829987,300.829987,18987000
2023-09-26,297.660004,300.299988,296.010010,298.959991,298.959991,19417200
2023-09-27,300.450012,301.299988,286.790009,297.739990,297.739990,36429800


In [5]:
def data_generator(data, seq_train, seq_label):
    data = data[['Open', 'Close']]
    data_ai = np.empty((len(data), 0, 2))
    for i in range(seq_train + seq_label - 1, -1, -1):
        data_ai = np.append(data_ai, np.array(data.shift(i))[np.newaxis,:,:].transpose((1, 0, 2)), axis=1)

    data_ai = data_ai[seq_train + seq_label - 1:]

    return data_ai[:, :seq_train, :], tf.transpose(data_ai[:, -seq_label:, :], perm = (0, 2, 1))

In [6]:
def inv_scale(x, min_max):
    return x * (min_max[1] - min_max[0]) + min_max[0]

In [7]:
train, label = data_generator(df, 50, 3)

In [8]:
train.shape

(1183, 50, 2)

In [10]:
label.shape

TensorShape([1183, 2, 3])

In [11]:
test_train = np.concatenate([np.array(df)[-50:][None, :, :], np.array(df)[-51:-1][None, :, :]], axis=0)
test_label = np.concatenate([np.array(df)[-3:][None, :, :], np.array(df)[-4:-1][None, :, :]], axis=0)

In [10]:
test_label = test_label.transpose((0, 2, 1))

In [11]:
class PositionEmbedding(tf.keras.layers.Layer):
    def __init__(self,*, seq_length, d_model, patch_size, strides, dff):
        super().__init__()
        self.seq_length = seq_length
        self.d_model = d_model
        self.patch_size = patch_size
        self.strides = strides
        self.num_patch = (seq_length - patch_size) // strides + 1
        self.MLP_P2D = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation="relu"),
            tf.keras.layers.Dense(d_model, activation="linear")
        ])
        self.MLP_Pos = tf.keras.Sequential([
            tf.keras.layers.Dense(d_model, activation="tanh"),
            tf.keras.layers.Dense(dff, activation="relu"),
            tf.keras.layers.Dense(d_model, activation="linear")
        ])
        
    def build(self, input_shape):
        self.position = self.add_weight(name="position", shape=(input_shape[-1], self.num_patch),   
                                 initializer=tf.initializers.Constant(tf.stack([tf.range(1., self.num_patch + 1)] * input_shape[-1])),
                                 trainable=False)

    def call(self, x):
        patches = tf.expand_dims(x[:, 0:self.patch_size, :], axis=0)
        for i in range(1, self.num_patch):
            patches = tf.concat([patches, tf.expand_dims(x[:, i * self.strides : i * self.strides + self.patch_size ,:], axis=0)], axis=0)
        x = tf.transpose(patches, perm = (1, 3, 0, 2))
        x = self.MLP_P2D(x)
        position_embedding = self.MLP_Pos(self.position[:,:,tf.newaxis])      
        x = tf.add(x, position_embedding)
        return x

In [12]:
class ClsCompute(tf.keras.layers.Layer):
    def __init__(self,*, d_model):
        super().__init__()
        self.cls_compute = tf.keras.layers.Conv1D(filters=d_model, kernel_size=1, padding="same", strides=1, activation="sigmoid")

    def call(self, x):
        x = tf.reshape(x, [-1 , x.shape[-3], x.shape[-2] * x.shape[-1]])
        x = self.cls_compute(x)
        return x

In [13]:
class Tokenize(tf.keras.layers.Layer):
    def __init__(self,*, seq_length, d_model, patch_size, strides, dff):
        super().__init__()
        self.pos_emb = PositionEmbedding(seq_length=seq_length, d_model=d_model, patch_size=patch_size, strides=strides, dff=dff)
        self.cls_tokenize = ClsCompute(d_model=d_model)

    def call(self, x):
        x = self.pos_emb(x)
        cls = tf.expand_dims(self.cls_tokenize(x), axis=2)
        return tf.concat([cls, x], axis=2)

In [14]:
tok = Tokenize(seq_length=50, d_model=128, patch_size=3, strides=1, dff=512)

In [15]:
toked = tok(train)

In [16]:
toked.shape

TensorShape([1183, 2, 49, 128])

In [17]:
import collections
import math
import string

import numpy as np
import tensorflow.compat.v2 as tf

from keras.src import constraints
from keras.src import initializers
from keras.src import regularizers
from keras.src.engine.base_layer import Layer
from keras.src.layers import activation
from keras.src.layers import core
from keras.src.layers import regularization
from keras.src.utils import tf_utils

# isort: off
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util.tf_export import keras_export
import tensorflow as tf
from keras import backend as K

In [18]:
import tensorflow as tf

class EMA_Layer(tf.keras.layers.Layer):
    def __init__(self, alpha=0.1, **kwargs):
        super(EMA_Layer, self).__init__(**kwargs)
        self.alpha = alpha

    def call(self, x):
        split = tf.split(x, x.shape[-2], axis=-2)
        ema_tensor = split[0]
        for i in range(1, x.shape[-2]):
            ema_tensor = tf.concat([ema_tensor, split[i] * self.alpha + ema_tensor[... , -1:, :] * (1 - self.alpha)], axis=-2) 
        
        return ema_tensor


In [19]:
import tensorflow as tf

class CustomSoftmax(tf.keras.layers.Layer):
    def __init__(self, axis=[-2, -1], **kwargs):
        super(CustomSoftmax, self).__init__(**kwargs)
        self.axis = axis

    def call(self, x):
        e = tf.exp(x)
        sum = tf.expand_dims(tf.expand_dims(tf.reduce_sum(e, axis=self.axis), axis=-1), axis=-1)
        return e / sum


In [77]:
class DualAttentionoverTokens(tf.keras.layers.MultiHeadAttention):
    def __init__(self, num_heads: int, key_dim: int, num_patch: int, decay: int, **kwargs):
        super().__init__(num_heads, key_dim, **kwargs)
        self.num_patch = num_patch
        self._MLP_query = tf.keras.Sequential([
            tf.keras.layers.Dense(num_patch * num_heads, activation="gelu"),
            tf.keras.layers.Dense(key_dim, activation="linear")
        ])
        self._MLP_key = tf.keras.Sequential([
            tf.keras.layers.Dense(num_patch * num_heads, activation="gelu"),
            tf.keras.layers.Dense(key_dim, activation="linear")
        ])
        self._MLP_value = tf.keras.Sequential([
            tf.keras.layers.Dense(num_patch * num_heads, activation="gelu"),
            tf.keras.layers.Dense(key_dim, activation="linear")
        ])
        self.EMA = EMA_Layer(alpha=decay)
        self.softmax = CustomSoftmax()
        self._MLP_projection = tf.keras.Sequential([
            tf.keras.layers.Dense(num_patch * num_heads, activation="linear"),
            tf.keras.layers.Dense(key_dim, activation="gelu"),
            tf.keras.layers.Dense(num_patch * num_heads, activation="linear"),
            tf.keras.layers.Dense(key_dim, activation="linear")
        ])
        self._MLP_projection1 = tf.keras.Sequential([
            tf.keras.layers.Dense(num_patch * num_heads, activation="linear"),
            tf.keras.layers.Dense(key_dim, activation="gelu"),
            tf.keras.layers.Dense(num_patch * num_heads, activation="linear"),
            tf.keras.layers.Dense(key_dim, activation="linear")
        ])
        self.add_norm = tf.keras.Sequential([
            tf.keras.layers.Add(),
            tf.keras.layers.LayerNormalization()
        ])

    def _build_from_signature(self, query, value, key=None):
        self._built_from_signature = True
        if hasattr(query, "shape"):
            self._query_shape = tf.TensorShape(query.shape)
        else:
            self._query_shape = tf.TensorShape(query)

        # Any setup work performed only once should happen in an `init_scope`
        # to avoid creating symbolic Tensors that will later pollute any eager
        # operations.
        with tf_utils.maybe_init_scope(self):
            free_dims = self._query_shape.rank - 1
            self._query_dense = core.EinsumDense(
                "bctd,chtd->bchtd",
                output_shape= [self._query_shape[1], self._num_heads, self.num_patch, self._key_dim],
                bias_axes="htd",
                name="query",
                **self._get_common_kwargs_for_sublayer(),
            )
            self._key_dense = core.EinsumDense(
                "bctd,chtd->bchtd",
                output_shape= [self._query_shape[1], self._num_heads, self.num_patch, self._key_dim],
                bias_axes="htd",
                name="key",
                **self._get_common_kwargs_for_sublayer(),
            )
            self._value_dense = core.EinsumDense(
                "bctd,chtd->bchtd",
                output_shape= [self._query_shape[1], self._num_heads, self.num_patch, self._key_dim],
                bias_axes="htd",
                name="value",
                **self._get_common_kwargs_for_sublayer(),
            )

            # Builds the attention computations for multi-head dot product
            # attention.  These computations could be wrapped into the keras
            # attention layer once it supports mult-head einsum computations.
            self._build_attention(6)
            self._output_dense = self._make_output_dense(
                self._get_common_kwargs_for_sublayer(),
                "attention_output",
            )
    #(5, 8, 3, 49, 128)
    def _make_output_dense(self, common_kwargs, name=None):
        return core.EinsumDense(
            "bchtd,hctd->bctd", 
            output_shape=[None] + self._query_shape[2:], 
            bias_axes='d',
            name=name,
            **common_kwargs,
        )

    def _compute_attention(
        self, query, key, value, attention_mask=None, training=None
    ):
        attention_scores1 = tf.einsum("...NK,...NO->...KO", query, key) / tf.sqrt(tf.cast(self.num_patch, tf.float32))
        attention_scores1 = self.softmax(attention_scores1)
        
        query = self.EMA(query)
        key = self.EMA(key)

        attention_scores = tf.einsum("...HCKO,...HCNO->...HCKN", query, key) / tf.sqrt(tf.cast(self._key_dim, tf.float32))
        attention_scores = self.softmax(attention_scores)
        
        attention_scores_dropout = self._dropout_layer(
            attention_scores, training=training
        )
        attention_scores1_dropout = self._dropout_layer(
            attention_scores1
        )

        # `context_layer` = [B, T, N, H]
        attention_output = tf.einsum(
            "...HCKN,...HCNO->...HCKO", attention_scores_dropout, value
        )
        attention_output1 = tf.einsum(
            "...HCDK,...HCNK->...HCND", attention_scores1_dropout, value
        )

        attention_output = self._MLP_projection(attention_output)
        attention_output1 = self._MLP_projection1(attention_output1)

        output = self.add_norm([attention_output, attention_output1])
        return output, attention_output, attention_output1

    def call(
        self,
        query,
        value,
        key=None,
        attention_mask=None,
        return_attention_scores=False,
        training=None,
    ):
        if not self._built_from_signature:
            self.num_patch = query.shape[2]
            self._build_from_signature(query=query, value=value, key=key)
        if key is None:
            key = value

        query = self._MLP_query(query)

        key = self._MLP_key(key)

        value = self._MLP_value(value)

        query = self._query_dense(query)

        key = self._key_dense(key)

        value = self._value_dense(value)

        attention_output, attention_scores, attention_scores1 = self._compute_attention(
            query, key, value, attention_mask, training
        )
    
        attention_output = self._output_dense(attention_output)
        
        if return_attention_scores:
            return attention_output, attention_scores, attention_scores1
        return attention_output

In [78]:
DAT = DualAttentionoverTokens(num_heads=8, key_dim=128, num_patch=49, decay=0.6)

In [79]:
DATed = DAT(query=toked[:5], key=toked[:5], value=toked[:5])

In [86]:
tf.reduce_sum(tf.cast(tf.math.is_nan(DATed), tf.int32))

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [90]:
class DualAttentionoverChannels(DualAttentionoverTokens):
    def __init__(self, num_heads: int, key_dim: int, num_patch: int, decay: int, **kwargs):
        super().__init__(num_heads, key_dim, num_patch, decay, **kwargs)
        self.DP_K = tf.keras.Sequential([
            tf.keras.layers.Dense(num_patch * num_heads, activation="linear"),
            tf.keras.layers.Dense(tf.math.ceil(key_dim / 10), activation="linear")
        ])
        self.DP_V = tf.keras.Sequential([
            tf.keras.layers.Dense(num_patch * num_heads, activation="linear"),
            tf.keras.layers.Dense(tf.math.ceil(key_dim / 10), activation="linear")
        ])

    def _compute_attention(
        self, query, key, value, attention_mask=None, training=None
    ):
        attention_scores1 = tf.einsum("...NK,...NO->...KO", query, key) / tf.sqrt(tf.cast(self.num_patch, tf.float32))
        attention_scores1 = self.softmax(attention_scores1)

        query = self.EMA(query)

        PK = self.DP_K(key)
        PV = self.DP_V(value)

        PK = self.softmax(PK)
        PV = self.softmax(PV)

        key = tf.einsum("...CNR,...CND->...RND", PK, key)
        key = self.EMA(key)

        value = tf.einsum("...CNR,...CND->...RND", PV, value)

        attention_scores = tf.einsum("...CKO,...RNO->...CKN", query, key) / tf.sqrt(tf.cast(self._key_dim, tf.float32))
        attention_scores = self.softmax(attention_scores)
        
        attention_scores_dropout = self._dropout_layer(
            attention_scores, training=training
        )
        attention_scores1_dropout = self._dropout_layer(
            attention_scores1
        )

        # `context_layer` = [B, T, N, H]
        attention_output = tf.einsum(
            "...CKN,...RND->...CKD", attention_scores_dropout, value
        )
        
        attention_output1 = tf.einsum(
            "...CDK,...RNK->...CND", attention_scores1_dropout, value
        )

        attention_output = self._MLP_projection(attention_output)
        attention_output1 = self._MLP_projection1(attention_output1)

        output = self.add_norm([attention_output, attention_output1])
        return output, attention_output, attention_output1

In [91]:
DAC = DualAttentionoverChannels(num_heads=8, key_dim=128, num_patch=49, decay=0.6)

In [93]:
k = DAC(query=toked[:5], key=toked[:5], value=toked[:5])

In [95]:
k.shape

TensorShape([5, 2, 49, 128])

In [265]:
class AttentionOverTokenDimension(tf.keras.layers.Layer):
    def __init__(
        self,
        channel: int,
        num_heads: int,
        key_dim: int,
        num_patch: int,
        decay: float,
        **kwargs,
    ):
        self.channel = channel
        super().__init__()
        self.list_DAC = [
            DualAttentionoverChannels(num_heads=num_heads, key_dim=key_dim, num_patch=num_patch, decay=decay)
            for _ in range(channel)
        ]
    
    def call(self, x):
        output = self.list_DAC[0](query=x[:, 0:1, :, :], key=x[:, 0:1, :, :], value=x[:, 0:1, :, :])
        for i in range(1, self.channel):
            output = tf.concat([output, self.list_DAC[i](query=x[:, i:i+1, :, :], key=x[:, i:i+1, :, :], value=x[:, i:i+1, :, :])], axis=1)
        return output

In [266]:
class AttentionOverChannelDimension(tf.keras.layers.Layer):
    def __init__(
        self,
        num_patch: int,
        num_heads: int,
        key_dim: int,
        decay: float,
        **kwargs,
    ):
        super().__init__()
        self.num_patch = num_patch
        self.list_DAT = [
            DualAttentionoverTokens(num_heads=num_heads, key_dim=key_dim, num_patch=num_patch, decay=decay)
            for _ in range(num_patch)
        ]
    
    def call(self, x):
        output = self.list_DAT[0](query=x[:, :, 0:1, :], key=x[:, :, 0:1, :], value=x[:, :, 0:1, :])
        for i in range(1, self.num_patch):
            output = tf.concat([output, self.list_DAT[i](query=x[:, :, i:i+1, :], key=x[:, :, i:i+1, :], value=x[:, :, i:i+1, :])], axis=2)
        return output

In [267]:
class DualTransformerBlock(tf.keras.layers.Layer):
    def __init__(
        self,
        seq_length: int,
        channel: int,
        patch_size: int,
        strides: int,
        dff: int,
        num_heads: int,
        d_model: int,
        decay: float,
        **kwargs,
    ):
        super().__init__()
        self.tokenize = Tokenize(seq_length=seq_length, d_model=d_model, patch_size=patch_size, strides=strides, dff=dff)
        self.attentionchannel = AttentionOverChannelDimension(num_patch=self.tokenize.pos_emb.num_patch + 1, num_heads=num_heads, key_dim=d_model, decay=decay)
        self.attentiontoken = AttentionOverTokenDimension(channel=channel, num_heads=num_heads, key_dim=d_model, num_patch=self.tokenize.pos_emb.num_patch + 1, decay=decay)
        self.add_norm = tf.keras.Sequential([
            tf.keras.layers.Add(),
            tf.keras.layers.LayerNormalization()
        ])
    
    def call(self, x):
        x = self.tokenize(x=x)
        x = self.attentionchannel(x)
        x_t = self.attentiontoken(x)
        x = self.add_norm([x, x_t])

        return x

In [268]:
class Tensor2Seq(tf.keras.layers.Layer):
    def __init__(self,*, output_length, channel, d_model, dff):
        super().__init__()
        self.channel = channel
        self.MLP = [tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(dff, activation="relu"),
            tf.keras.layers.Dense(d_model, activation="gelu"),
            tf.keras.layers.Dense(output_length, activation="linear")
        ]) for _ in range(channel)]

    def call(self, x):
        output = [
            self.MLP[i](x[:,i,:,:])
            for i in range(self.channel)
        ]
        return output

In [269]:
class Transformer(tf.keras.Model):
    def __init__(
        self,
        seq_length: int,
        output_length: int,
        channel: int,
        patch_size: int,
        strides: int,
        dff: int,
        num_heads: int,
        d_model: int,
        decay: float,
        **kwargs,
    ):
        super().__init__()
        self.TransformerBlock = DualTransformerBlock(seq_length=seq_length, channel=channel, patch_size=patch_size, strides=strides, dff=dff, num_heads=num_heads, d_model=d_model, decay=decay)
        self.T2S = Tensor2Seq(output_length=output_length, channel=channel, d_model=d_model, dff=dff)

    def call(self, x):
        
        x = self.TransformerBlock(x)
        x = self.T2S(x)

        return x

In [270]:
transformer = Transformer(50, 3, 3, 3, 1, 10, 8, 128, 0.6)

In [54]:
np.exp(196)

1.3234832615645704e+85

In [276]:
transformer.predict(u[0][:1])

1/1 [==============================] - 1s 765ms/step


[array([[ 0.16109133, -0.0156713 , -0.3824173 ]], dtype=float32),
 array([[-0.73473114,  0.90678895,  0.4228015 ]], dtype=float32),
 array([[ 0.35994428, -0.5369685 , -0.2902751 ]], dtype=float32)]

In [277]:
transformer.summary()

Model: "transformer_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dual_transformer_block_6 (  multiple                  40625558  
 DualTransformerBlock)                                           
                                                                 
 tensor2_seq_6 (Tensor2Seq)  multiple                  193575    
                                                                 
Total params: 40819133 (155.71 MB)
Trainable params: 40818989 (155.71 MB)
Non-trainable params: 144 (576.00 Byte)
_________________________________________________________________


In [ ]:
transformer.save("C:/Users/Admin/Downloads/model")

KeyboardInterrupt: 

In [279]:
transformer.load_weights("C:/Users/Admin/Downloads/model.h5")

In [50]:
from tensorflow.keras import backend as K

def signal_decay_loss(y_true, y_pred):
    length = len(y_true)
    seq_length = y_true.shape[-1]
    loss = K.sum(K.abs(y_true - y_pred), axis=0)
    decay = tf.range(1, seq_length + 1, dtype=tf.float64)
    decay = K.pow(decay, -0.5)
    loss = K.min(loss / decay / length)
    return loss


In [51]:
signal_decay_loss(t[:, 0, :], t[:, 1, :])

<tf.Tensor: shape=(), dtype=float64, numpy=3.1567707980777486>

In [280]:
transformer.compile(optimizer="adam", loss="mse", metrics=["mse"])

In [281]:
transformer.fit(x=u[0][:5], y=[u[1][:5][:,:,0], u[1][:5][:,:,1], u[1][:5][:,:,2]], epochs=100)

Epoch 1/100
1/1 [==============================] - 801s 801s/step - loss: 519930957004800.0000 - output_1_loss: 21945.5430 - output_2_loss: 21691.3398 - output_3_loss: 519930957004800.0000 - output_1_mse: 21945.5430 - output_2_mse: 21691.3398 - output_3_mse: 519930957004800.0000
Epoch 2/100
1/1 [==============================] - 9s 9s/step - loss: 519930923450368.0000 - output_1_loss: 21697.5410 - output_2_loss: 21380.6973 - output_3_loss: 519930923450368.0000 - output_1_mse: 21697.5410 - output_2_mse: 21380.6973 - output_3_mse: 519930923450368.0000
Epoch 3/100
1/1 [==============================] - 8s 8s/step - loss: 519930554351616.0000 - output_1_loss: 20756.8691 - output_2_loss: 19943.5781 - output_3_loss: 519930554351616.0000 - output_1_mse: 20756.8691 - output_2_mse: 19943.5781 - output_3_mse: 519930554351616.0000
Epoch 4/100
1/1 [==============================] - 7s 7s/step - loss: nan - output_1_loss: 19894.6094 - output_2_loss: 18722.5781 - output_3_loss: nan - output_1_mse: 1

KeyboardInterrupt: 

In [839]:
transformer.predict(u[0][0][None, :, :])

(5, 3, 49, 128)
1/1 [==============================] - 116s 116s/step


[array([[-0.34613574, -0.24725723,  0.22485566],
        [-0.24567108, -0.24268132, -0.44509122],
        [-0.42449155,  0.12960796, -0.8861203 ],
        [-0.10334237,  0.03639297,  0.01217689],
        [-0.16685176,  0.09116859, -0.3668781 ]], dtype=float32),
 array([[-0.52959263, -0.31515434,  0.29144627],
        [-0.00625029,  0.03018725,  0.00985443],
        [ 0.14792624,  0.00618448, -0.48661834],
        [-0.19680572, -0.31270456,  0.1877094 ],
        [-0.04569163, -0.3061453 , -0.11313421]], dtype=float32),
 array([[ 0.7568802 ,  0.3032534 ,  0.35585985],
        [ 0.57560945,  0.0543299 ,  0.17535202],
        [-0.13026804, -0.28158945, -0.6424637 ],
        [ 0.63206834, -0.3839475 , -0.18032119],
        [ 0.1314375 , -0.53101546, -0.95899737]], dtype=float32)]

In [807]:
h = transformer.TransformerBlock.tokenize(u[0][0][None, :, :])

In [840]:
transformer(u[0][0][None, :, :])

(5, 3, 49, 128)


[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-0.34613577, -0.24725717,  0.2248556 ],
        [-0.24567099, -0.24268141, -0.4450912 ],
        [-0.42449152,  0.12960795, -0.8861203 ],
        [-0.10334238,  0.03639295,  0.01217688],
        [-0.16685173,  0.09116861, -0.3668781 ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-0.52959275, -0.31515425,  0.29144633],
        [-0.00625029,  0.03018718,  0.00985442],
        [ 0.1479263 ,  0.00618459, -0.48661837],
        [-0.19680575, -0.31270462,  0.1877094 ],
        [-0.04569162, -0.3061453 , -0.11313421]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 0.7568803 ,  0.30325335,  0.35585988],
        [ 0.5756095 ,  0.05432988,  0.17535204],
        [-0.13026804, -0.28158942, -0.64246356],
        [ 0.6320684 , -0.38394752, -0.18032119],
        [ 0.13143738, -0.53101546, -0.9589976 ]], dtype=float32)>]

In [282]:
def signal_decay_loss(y_true, y_pred):
    L = len(y_true)
    loss = 0
    for l in range(1, L+1):
        loss += (l**-0.5) * tf.abs(y_pred[l-1] - y_true[l-1])
    return loss / L


In [ ]:
signal_decay_loss(u[1][])

In [7]:
t = tf.transpose(u[1], perm=(0, 2, 1))

In [304]:
u[0][3]

array([[1.49309998e+02, 1.49940002e+02, 1.66671000e+07],
       [1.51570007e+02, 1.51529999e+02, 2.18774000e+07],
       [1.50490005e+02, 1.47869995e+02, 2.41458000e+07],
       [1.46750000e+02, 1.44960007e+02, 1.73269000e+07],
       [1.44479996e+02, 1.41550003e+02, 1.85421000e+07],
       [1.42000000e+02, 1.42160004e+02, 1.51417000e+07],
       [1.43699997e+02, 1.44220001e+02, 2.20684000e+07],
       [1.42330002e+02, 1.43850006e+02, 3.03203000e+07],
       [1.41070007e+02, 1.39529999e+02, 3.72506000e+07],
       [1.37610001e+02, 1.31550003e+02, 4.43627000e+07],
       [1.27029999e+02, 1.32429993e+02, 4.19395000e+07],
       [1.34399994e+02, 1.34820007e+02, 2.54697000e+07],
       [1.33649994e+02, 1.31729996e+02, 1.18861000e+07],
       [1.33000000e+02, 1.36380005e+02, 2.42636000e+07],
       [1.35750000e+02, 1.35000000e+02, 2.07503000e+07],
       [1.36279999e+02, 1.36759995e+02, 2.98475000e+07],
       [1.35919998e+02, 1.38679993e+02, 2.42387000e+07],
       [1.38259995e+02, 1.40610

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[1.49000000e+02, 1.46949997e+02, 1.49750000e+02],
       [1.47539993e+02, 1.48300003e+02, 1.50039993e+02],
       [1.80257000e+07, 1.57879000e+07, 3.10296000e+07]])>

In [293]:
u[0]

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[1.49000000e+02, 1.46949997e+02, 1.49750000e+02],
       [1.47539993e+02, 1.48300003e+02, 1.50039993e+02],
       [1.80257000e+07, 1.57879000e+07, 3.10296000e+07]])>

In [7]:
126**53%161

42